## Aplicando ResNet

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [22]:
df1 = pd.read_excel('wclassesFiltered.xlsx')

In [23]:
for img_p in range(df1.shape[0]):
    img_path = df1.iloc[img_p, 1]
    if "Ã¢Â\xa0" in img_path:
        img_path = img_path.replace('Ã¢Â', '\u00E2')
        temp_path = img_path.replace('\u00E2', 'a') 
        df1.iat[img_p, 1] = temp_path
    df1.iat[img_p, 4] = df1.iloc[img_p, 4]-140

In [24]:
df1.loc[:,'style'] = df1['style'].astype(str)

In [25]:
train = df1.sample(frac=0.8,random_state=200)
test = df1.drop(train.index)

In [26]:
target_size=(256,256) # set the size of the images
color_mode='rgb' # set the type of image
class_mode= 'categorical' # set the class mode
batch_size=64  # set the batch size
subset='training' # set to 'training', or 'valiatiom' or leave as None

train_gen = ImageDataGenerator(validation_split=0.2)
test_gen = ImageDataGenerator()
val_gen = ImageDataGenerator()

train_imgs= train_gen.flow_from_dataframe(dataframe=train, 
          x_col='file',
          y_col='style',
          target_size=target_size,color_mode=color_mode,
          class_mode=class_mode, batch_size=batch_size,shuffle=True, seed=123,
          subset='training')
val_imgs= train_gen.flow_from_dataframe(dataframe=train, 
          x_col='file',
          y_col='style',
          target_size=target_size,color_mode=color_mode,
          class_mode=class_mode, batch_size=batch_size,shuffle=True, seed=123,
          subset="validation")
test_imgs= test_gen.flow_from_dataframe(dataframe=test, 
          x_col='file',
          y_col='style',
          target_size=target_size,color_mode=color_mode,
          class_mode=class_mode, batch_size=batch_size,shuffle=True, seed=123,
          subset='training')

Found 5120 validated image filenames belonging to 9 classes.
Found 1280 validated image filenames belonging to 9 classes.
Found 1600 validated image filenames belonging to 9 classes.


ResNet é uma ANN (artificial neural network), variante da HigwayNet (https://en.wikipedia.org/wiki/Highway_network). Pode possuir centenas de camadas, sendo muito mais profunda do que redes neurais anteriores. São usados conexões de salto ou atalhos ​​para saltar sobre algumas camadas. Modelos ResNet típicos são implementados com saltos de camada dupla ou tripla que contêm não linearidades (ReLU) e normalização de lote no meio. Modelos com vários saltos paralelos são referidos como DenseNets.

<img src="File_ResNets.png" style="height: 300px;"/>   

Forma canônica de uma rede neural residual. Uma camada ℓ   − 1 é ignorada na ativação de ℓ  − 2."

#### Arquitetura ResNet:

<img src="arquitetura_resnet.png" style="height: 350px;"/>   

_ResNet de 34 camadas com conexão Skip/Atalho (superior), rede simples de 34 camadas (meio), VGG-19 de 19 camadas (inferior)_

O VGG-19 [2] (abaixo na imagem) é uma abordagem de última geração no ILSVRC 2014.
A rede plana de 34 camadas (no meio) é tratada como a rede mais profunda do VGG-19 , ou seja, camadas mais convolucionais.
A rede residual de 34 camadas (ResNet) (topo) é a simples com adição de conexão skip/atalho.

A ResNet usada é de grande profundidade, contendo **152** camadas.

In [37]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (256,256,3))
for layer in base_model.layers:
  layer.trainable = False

In [38]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(9, activation = 'softmax')(x)

In [39]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [40]:
history = head_model.fit(train_imgs, epochs=40, validation_data=(val_imgs))

Epoch 1/40
80/80 [==============================] - 36s 350ms/step - loss: 32.9757 - accuracy: 0.6670 - val_loss: 2.4264 - val_accuracy: 0.7453
Epoch 2/40
80/80 [==============================] - 25s 315ms/step - loss: 0.4822 - accuracy: 0.9131 - val_loss: 1.3212 - val_accuracy: 0.8281
Epoch 3/40
80/80 [==============================] - 25s 316ms/step - loss: 0.1040 - accuracy: 0.9787 - val_loss: 1.3569 - val_accuracy: 0.8062
Epoch 4/40
80/80 [==============================] - 25s 316ms/step - loss: 0.0468 - accuracy: 0.9936 - val_loss: 1.3235 - val_accuracy: 0.8375
Epoch 5/40
80/80 [==============================] - 25s 315ms/step - loss: 0.0563 - accuracy: 0.9957 - val_loss: 1.3732 - val_accuracy: 0.8359
Epoch 6/40
80/80 [==============================] - 25s 316ms/step - loss: 0.0459 - accuracy: 0.9969 - val_loss: 1.3113 - val_accuracy: 0.8359
Epoch 7/40
80/80 [==============================] - 25s 316ms/step - loss: 0.0355 - accuracy: 0.9965 - val_loss: 1.3489 - val_accuracy: 0.835

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = head_model.evaluate(test_imgs, verbose=2)

In [ ]:
#head_model.evaluate(x_test, y_test)